# Segmenting and Clustering Neighborhoods in Toronto

## Packages

### Install Packages

In [1]:
#pip install beautifulsoup4
#pip install lxml
#pip install requests

### Import Packages

In [54]:
from bs4 import BeautifulSoup as bsoup
import requests as req
import csv
import pandas as pd
import numpy as np

## Data

In [88]:
#Get information from Wikipedia

source = req.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bsoup(source, 'lxml')
#print(soup.prettify())

# make the first table in the page into a variable
table = soup.table
#table

## Create the DataFrame

### Create Headers

In [102]:
# Get headers from table
row_th = table.find_all('th')

# Make them into a string, and clean out the html code, the brackets, etc.
str_headers = str(row_th)
cleanheader = bsoup(str_headers, "lxml").get_text()
cleanheader = cleanheader.replace("[","")
cleanheader = cleanheader.replace("\n]","")

# Make the resulting string into a list
cleanheader = list(cleanheader.split(", "))

### Create Rows

In [101]:
# Get all the rows in that table, separately
rows = table.find_all('tr')
#print(rows)

# Make an empty variable
list_rows = []

# Fill the variable with each table row, removing brackets, etc., and making it into a list
for row in rows:
    row_td = row.find_all('td')
    str_cells = str(row_td)
    cleantext = bsoup(str_cells, "lxml").get_text()
    cleantext = cleantext.replace("[","")
    cleantext = cleantext.replace("\n]","")
    cleantext = list(cleantext.split(", "))
    list_rows.append(cleantext)
#print(cleantext)

# Put each row into a row in a dataframe
df = pd.DataFrame(list_rows)
df = df.drop(0)  ## drop the top row, which belongs to the 'th' headers

### Assign Column Names and Drop Missing Boroughs

In [103]:
df.columns = cleanheader
df.head()

# drop "Not assigned" boroughs
df = df[df.Borough != 'Not assigned']

### Rename Missing Neighbourhoods with Borough Name

In [92]:
values = df['Borough']
df['Neighbourhood'] = values.where(df.Neighbourhood == "Not assigned", other=df.Neighbourhood)

### Combine Multiple Neighborhoods in Postcode

In [97]:
df = df.groupby(['Postcode','Borough'],sort=False).agg(', '.join).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [98]:
df.to_csv(r'torontosegs.csv')

In [99]:
df.shape

(103, 3)